In [ ]:
# imports
import pandas as pd

In [ ]:
# get the intents dataset
SQL = "SELECT * from `datasets.airline-intents`"
PROJ = "questrom"


intents = pd.read_gbq(SQL, PROJ)

In [ ]:
# reminder of what we have
intents.intent.value_counts()

In [ ]:
### sourced from here: https://github.com/lohithpro/csvtorasa/blob/master/csvtorasa.py
## I tweaked below for colab

def create_rasa_files(path, create_files_path):
    """
    Converts an CSV file created with the specified format to RASA accepted nlu.md format
    :param path: path where the CSV file is present
    :param create_files_path: path where the nlu.md file needs to be created
    :return: return nothing. A file is created in the path specified via create_files_path
    """
    df = pd.read_csv(r"{}".format(path))
    file = open(r'{}/nlu.md'.format(create_files_path), "w")
    intents = list(df.columns)
    for item in intents:
        file.write("## intent: {intent_name}\n".format(intent_name=item))
        for sent in df[item]:
            file.write("- {}\n".format(sent))
    file.close()
    file = open(r'{}/domain.yml'.format(create_files_path), "w")
    file.write("intents:\n")
    for intent_name in intents:
        file.write("  - {}\n".format(intent_name))
    file.write("actions:\n")
    for intent_name in intents:
        file.write("  - utter_{}\n".format(intent_name))
    file.close()
    return None

In [ ]:
# I would have expected to actually parse a dataframe
# but its easy enough I suppose


In [ ]:
############################## only keep intents with a # of records
# 20 examples per intent, more, in theory, better

# how many intents to keep based on frequency
THRESH = 20
F = intents.intent.value_counts() > THRESH
i2k = intents.intent.value_counts()[F]

top_intents = i2k.index.to_list()

## filter the list of intents to those with the threshold volume
intents2 = intents.loc[intents.intent.isin(top_intents), :]

In [ ]:
# for each of the top intents, build a dictionary of intent:example pairs
intent_d = {}
for intent in top_intents:
  tmp = intents2.loc[intents2.intent==intent, :]
  examples = tmp.text.tolist()
  intent_d[str(intent)] = examples


# intent_d.keys()


# make it a dataframe
intent_nlu = pd.DataFrame.from_dict(intent_d, orient='index').transpose()



In [ ]:
# type(intent_nlu)

In [ ]:
# shape
intent_nlu.shape

In [ ]:
# first few rows
intent_nlu.head(5)

In [ ]:
# not all rows are valid per intent
# missing is fine

# write the file
intent_nlu.to_csv("intents.csv", index=False)

In [ ]:
# use the nlu function
create_rasa_files("intents.csv", "/content/")

In [ ]:
# what do we have now
! ls -l

In [ ]:
# 